In [23]:
%%capture
!pip install gradio beautifulsoup4 yfinance torch

In [24]:
import gradio as gr
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle
import requests
from bs4 import BeautifulSoup
import re
import io
from PIL import Image
import math
from datetime import datetime, timedelta

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

DEBUG = True
def debug_print(msg):
    if DEBUG:
        print(msg)

In [25]:
# FOMC Dates Scraper
def get_fomc_dates(start_date, end_date):
    url = "https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code != 200:
            debug_print(f"Error: Received status code {response.status_code}")
            return []
        soup = BeautifulSoup(response.text, "html.parser")
        date_objs = []
        for text in soup.stripped_strings:
            matches = re.findall(r'([A-Za-z]+ \d{1,2}, \d{4})', text)
            for date_str in matches:
                try:
                    dt = datetime.strptime(date_str, "%B %d, %Y")
                    if dt not in date_objs:
                        date_objs.append(dt)
                except Exception:
                    continue
        date_objs = sorted(date_objs)
        start_dt = pd.to_datetime(start_date)
        end_dt = pd.to_datetime(end_date)
        filtered_dates = [dt for dt in date_objs if start_dt <= dt <= end_dt]
        return filtered_dates
    except Exception as e:
        debug_print(f"Error scraping FOMC dates: {e}")
        return []

In [26]:
# Indicator Functions
def exp_average(series, period):
    return series.ewm(span=period, adjust=False).mean()

def wilder_average(series, length):
    return series.ewm(alpha=1/length, adjust=False).mean()

def weighted_moving_average(series, window):
    weights = np.arange(1, window+1)
    return series.rolling(window).apply(lambda prices: np.dot(prices, weights)/weights.sum(), raw=True)

def t3(source, length=21, vf=0.7):
    ema1 = exp_average(source, length)
    ema2 = exp_average(ema1, length)
    gd1 = ema1*(1+vf) - ema2*vf
    ema11 = exp_average(gd1, length)
    ema22 = exp_average(ema11, length)
    gd2 = ema11*(1+vf) - ema22*vf
    ema111 = exp_average(gd2, length)
    ema222 = exp_average(ema111, length)
    gd3 = ema111*(1+vf) - ema222*vf
    return gd3

def vwma(series, window, volume):
    return (series*volume).rolling(window=window, min_periods=window).sum()/volume.rolling(window=window, min_periods=window).sum()

def rsi_function(close, sensitivity, rsiPeriod, rsiBase):
    delta = close.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=rsiPeriod, min_periods=rsiPeriod).mean()
    avg_loss = loss.rolling(window=rsiPeriod, min_periods=rsiPeriod).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100/(1+rs))
    rsi = rsi.fillna(50)
    rsi_adj = sensitivity*(rsi-rsiBase)
    return rsi_adj.clip(lower=0, upper=20)

def download_data(ticker, start_date, end_date):
    df = yf.download(ticker, start=pd.to_datetime(start_date), end=pd.to_datetime(end_date))
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [col[0].lower() for col in df.columns]
    else:
        df.columns = [str(col).lower() for col in df.columns]
    return df

def compute_bressert(df, n_period=8, r_period=13):
    df['Ln'] = df['low'].rolling(window=n_period, min_periods=1).min()
    df['Hn'] = df['high'].rolling(window=n_period, min_periods=1).max()
    df['Y'] = ((df['close']-df['Ln'])/(df['Hn']-df['Ln']))*100
    df['X'] = exp_average(df['Y'], r_period)
    df['Lxn'] = df['X'].rolling(window=n_period, min_periods=1).min()
    df['Hxn'] = df['X'].rolling(window=n_period, min_periods=1).max()
    df['DSS'] = ((df['X']-df['Lxn'])/(df['Hxn']-df['Lxn']))*100
    df['DSSb'] = exp_average(df['DSS'], r_period)
    df['DSSsignal'] = df['DSSb'].shift(1)
    return df

def compute_zscore(df, length_m=14):
    momentum = df['close'] - df['close'].shift(length_m)
    avgMomentum = momentum.rolling(window=length_m, min_periods=length_m).mean()
    stdDevMomentum = momentum.rolling(window=length_m, min_periods=length_m).std().fillna(0)
    zScore = (momentum - avgMomentum)/stdDevMomentum
    return zScore

def compute_zero_lag_macd(source, fastLength=12, slowLength=26, signalLength=9, MacdEmaLength=9, useEma=True, useOldAlgo=False):
    if useEma:
        ma1 = source.ewm(span=fastLength, adjust=False).mean()
        ma2 = ma1.ewm(span=fastLength, adjust=False).mean()
    else:
        ma1 = source.rolling(window=fastLength, min_periods=fastLength).mean()
        ma2 = ma1.rolling(window=fastLength, min_periods=fastLength).mean()
    zerolagEMA = (2*ma1) - ma2
    if useEma:
        mas1 = source.ewm(span=slowLength, adjust=False).mean()
        mas2 = mas1.ewm(span=slowLength, adjust=False).mean()
    else:
        mas1 = source.rolling(window=slowLength, min_periods=slowLength).mean()
        mas2 = mas1.rolling(window=slowLength, min_periods=slowLength).mean()
    zerolagslowMA = (2*mas1) - mas2
    ZeroLagMACD = zerolagEMA - zerolagslowMA
    emasig1 = ZeroLagMACD.ewm(span=signalLength, adjust=False).mean()
    emasig2 = emasig1.ewm(span=signalLength, adjust=False).mean()
    if useOldAlgo:
        signal = ZeroLagMACD.rolling(window=signalLength, min_periods=signalLength).mean()
    else:
        signal = (2*emasig1) - emasig2
    hist = ZeroLagMACD - signal
    upHist = hist.copy()
    upHist[hist <= 0] = 0
    downHist = hist.copy()
    downHist[hist > 0] = 0
    EMALine = ZeroLagMACD.ewm(span=MacdEmaLength, adjust=False).mean()
    dotUP = ZeroLagMACD.copy()
    dotUP[(ZeroLagMACD.shift(1) >= signal.shift(1)) | (ZeroLagMACD < signal)] = np.nan
    dotDN = ZeroLagMACD.copy()
    dotDN[(ZeroLagMACD.shift(1) <= signal.shift(1)) | (ZeroLagMACD > signal)] = np.nan
    return {
        "ZeroLagMACD": ZeroLagMACD,
        "signal": signal,
        "hist": hist,
        "upHist": upHist,
        "downHist": downHist,
        "EMALine": EMALine,
        "dotUP": dotUP,
        "dotDN": dotDN
    }

def compute_basic_macd(source, fast=12, slow=26, signal=9):
    ema_fast = source.ewm(span=fast, adjust=False).mean()
    ema_slow = source.ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    hist_line = macd_line - signal_line
    return {
        "basicMACD": macd_line,
        "signal": signal_line,
        "hist": hist_line
    }

In [27]:
# Historical Signals Extraction
def extract_signals(df, signalUp_ZLMA, signalDn_ZLMA, bullPt, bearPt,
                    upSig_MCDX, dnSig_MCDX, length_m=14):
    """
    Combines ZLMA, RSI, MCDX, DSS signals in a single table.
    """
    signals = []
    zScore = compute_zscore(df, length_m)

    # ZLMA
    for dt in df.index[signalUp_ZLMA.fillna(False)]:
        signals.append({
            "Date": dt.strftime("%Y-%m-%d"),
            "Signal": "ZLMA Buy",
            "Z-Score": round(zScore.loc[dt],2) if not pd.isna(zScore.loc[dt]) else None
        })
    for dt in df.index[signalDn_ZLMA.fillna(False)]:
        signals.append({
            "Date": dt.strftime("%Y-%m-%d"),
            "Signal": "ZLMA Sell",
            "Z-Score": round(zScore.loc[dt],2) if not pd.isna(zScore.loc[dt]) else None
        })
    # RSI
    if isinstance(bullPt, pd.Series):
        for dt in bullPt.dropna().index:
            signals.append({
                "Date": dt.strftime("%Y-%m-%d"),
                "Signal": "RSI Buy",
                "Z-Score": round(zScore.loc[dt],2) if not pd.isna(zScore.loc[dt]) else None
            })
    if isinstance(bearPt, pd.Series):
        for dt in bearPt.dropna().index:
            signals.append({
                "Date": dt.strftime("%Y-%m-%d"),
                "Signal": "RSI Sell",
                "Z-Score": round(zScore.loc[dt],2) if not pd.isna(zScore.loc[dt]) else None
            })
    # MCDX
    if isinstance(upSig_MCDX, pd.Series):
        for dt in upSig_MCDX.dropna().index:
            signals.append({
                "Date": dt.strftime("%Y-%m-%d"),
                "Signal": "MCDX Buy",
                "Z-Score": round(zScore.loc[dt],2) if not pd.isna(zScore.loc[dt]) else None
            })
    if isinstance(dnSig_MCDX, pd.Series):
        for dt in dnSig_MCDX.dropna().index:
            signals.append({
                "Date": dt.strftime("%Y-%m-%d"),
                "Signal": "MCDX Sell",
                "Z-Score": round(zScore.loc[dt],2) if not pd.isna(zScore.loc[dt]) else None
        })
    # DSS
    for i in range(1, len(df)):
        if (pd.notna(df['DSSb'].iloc[i]) and pd.notna(df['DSSsignal'].iloc[i]) and
            pd.notna(df['DSSb'].iloc[i-1]) and pd.notna(df['DSSsignal'].iloc[i-1])):
            if df['DSSb'].iloc[i] > df['DSSsignal'].iloc[i] and df['DSSb'].iloc[i-1] <= df['DSSsignal'].iloc[i-1]:
                dt = df.index[i]
                signals.append({
                    "Date": dt.strftime("%Y-%m-%d"),
                    "Signal": "DSS Buy",
                    "Z-Score": round(zScore.loc[dt],2) if not pd.isna(zScore.loc[dt]) else None
                })
            elif df['DSSb'].iloc[i] < df['DSSsignal'].iloc[i] and df['DSSb'].iloc[i-1] >= df['DSSsignal'].iloc[i-1]:
                dt = df.index[i]
                signals.append({
                    "Date": dt.strftime("%Y-%m-%d"),
                    "Signal": "DSS Sell",
                    "Z-Score": round(zScore.loc[dt],2) if not pd.isna(zScore.loc[dt]) else None
                })

    signals_df = pd.DataFrame(signals)
    if not signals_df.empty:
        signals_df["Date"] = pd.to_datetime(signals_df["Date"])
        signals_df = signals_df.sort_values("Date", ascending=False)
    return signals_df

def extract_momentum_signals(df, length_m=14):
    """
    Momentum-based signals. Checks momentum grade changes, direction changes, and state changes.
    """
    momentum = df['close'] - df['close'].shift(length_m)
    avgMomentum = momentum.rolling(window=length_m, min_periods=length_m).mean()
    stdDevMomentum = momentum.rolling(window=length_m, min_periods=length_m).std().fillna(0)
    zScore = (momentum - avgMomentum) / stdDevMomentum

    def grade(x):
        if x >= 2:
            return "A"
        elif x >= 1:
            return "B"
        elif x >= 0:
            return "C"
        elif x >= -1:
            return "D"
        elif x >= -2:
            return "E"
        else:
            return "F"

    momentum_grade = zScore.apply(grade)
    momentum_direction = momentum.apply(lambda x: "Increasing" if x>0 else "Decreasing")

    momentum_state = []
    for i in range(len(momentum)):
        if i == 0:
            momentum_state.append("N/A")
        else:
            if abs(momentum.iloc[i]) < abs(momentum.iloc[i-1]) * 0.0001:
                momentum_state.append("Consolidating")
            elif momentum.iloc[i] * momentum.iloc[i-1] < 0:
                if momentum.iloc[i] > 0:
                    momentum_state.append("Turning Up")
                else:
                    momentum_state.append("Turning Down")
            elif (momentum.iloc[i] > 0 and (momentum.iloc[i] - momentum.iloc[i-1]) < 0) or (momentum.iloc[i] < 0 and (momentum.iloc[i] - momentum.iloc[i-1]) > 0):
                momentum_state.append("Stalling")
            elif momentum.iloc[i] > 0:
                momentum_state.append("Positive Trending")
            else:
                momentum_state.append("Negative Trending")
    momentum_state = pd.Series(momentum_state, index=df.index)

    signals = []
    for i in range(1, len(df)):
        # Grade
        if momentum_grade.iloc[i] != momentum_grade.iloc[i-1]:
            signals.append({
                "Date": df.index[i].strftime("%Y-%m-%d"),
                "Signal": f"Momentum Grade Changed to {momentum_grade.iloc[i]}",
                "Z-Score": round(zScore.iloc[i],2)
            })
        # Direction
        if momentum_direction.iloc[i] != momentum_direction.iloc[i-1]:
            signals.append({
                "Date": df.index[i].strftime("%Y-%m-%d"),
                "Signal": f"Momentum Direction Changed to {momentum_direction.iloc[i]}",
                "Z-Score": round(zScore.iloc[i],2)
            })
        # State
        if momentum_state.iloc[i] != momentum_state.iloc[i-1]:
            signals.append({
                "Date": df.index[i].strftime("%Y-%m-%d"),
                "Signal": f"Momentum State Changed to {momentum_state.iloc[i]}",
                "Z-Score": round(zScore.iloc[i],2)
            })
    signals_df = pd.DataFrame(signals)
    if not signals_df.empty:
        signals_df["Date"] = pd.to_datetime(signals_df["Date"])
        signals_df = signals_df.sort_values("Date", ascending=False)
    return signals_df

def extract_zero_macd_signals(df, zero_macd_dict, length_m=14):
    macd_line = zero_macd_dict["ZeroLagMACD"]
    macd_mean = macd_line.rolling(window=length_m, min_periods=length_m).mean()
    macd_std = macd_line.rolling(window=length_m, min_periods=length_m).std().replace(0, np.nan)
    macd_zscore = (macd_line - macd_mean)/macd_std

    signals = []
    for i in range(1, len(df)):
        if (pd.notna(zero_macd_dict["ZeroLagMACD"].iloc[i]) and
            pd.notna(zero_macd_dict["signal"].iloc[i]) and
            pd.notna(zero_macd_dict["ZeroLagMACD"].iloc[i-1]) and
            pd.notna(zero_macd_dict["signal"].iloc[i-1])):
            dt = df.index[i]
            if zero_macd_dict["ZeroLagMACD"].iloc[i] > zero_macd_dict["signal"].iloc[i] and zero_macd_dict["ZeroLagMACD"].iloc[i-1] <= zero_macd_dict["signal"].iloc[i-1]:
                signals.append({
                    "Date": dt.strftime("%Y-%m-%d"),
                    "Signal": "ZeroLag MACD Buy",
                    "Z-Score": round(macd_zscore.iloc[i],2) if not pd.isna(macd_zscore.iloc[i]) else None
                })
            elif zero_macd_dict["ZeroLagMACD"].iloc[i] < zero_macd_dict["signal"].iloc[i] and zero_macd_dict["ZeroLagMACD"].iloc[i-1] >= zero_macd_dict["signal"].iloc[i-1]:
                signals.append({
                    "Date": dt.strftime("%Y-%m-%d"),
                    "Signal": "ZeroLag MACD Sell",
                    "Z-Score": round(macd_zscore.iloc[i],2) if not pd.isna(macd_zscore.iloc[i]) else None
                })
    signals_df = pd.DataFrame(signals)
    if not signals_df.empty:
        signals_df["Date"] = pd.to_datetime(signals_df["Date"])
        signals_df = signals_df.sort_values("Date", ascending=False)
    return signals_df

def extract_basic_macd_signals(df, basic_macd_dict, length_m=14):
    macd_line   = basic_macd_dict["basicMACD"]
    signal_line = basic_macd_dict["signal"]
    macd_mean   = macd_line.rolling(window=length_m, min_periods=length_m).mean()
    macd_std    = macd_line.rolling(window=length_m, min_periods=length_m).std().replace(0, np.nan)
    macd_zscore = (macd_line - macd_mean)/macd_std

    signals=[]
    for i in range(1, len(df)):
        if (pd.notna(macd_line.iloc[i]) and pd.notna(signal_line.iloc[i]) and
            pd.notna(macd_line.iloc[i-1]) and pd.notna(signal_line.iloc[i-1])):
            dt= df.index[i]
            # cross up
            if macd_line.iloc[i]> signal_line.iloc[i] and macd_line.iloc[i-1]<= signal_line.iloc[i-1]:
                signals.append({
                    "Date": dt.strftime("%Y-%m-%d"),
                    "Signal": "Basic MACD Buy",
                    "Z-Score": round(macd_zscore.iloc[i],2) if not pd.isna(macd_zscore.iloc[i]) else None
                })
            # cross down
            elif macd_line.iloc[i]< signal_line.iloc[i] and macd_line.iloc[i-1]>= signal_line.iloc[i-1]:
                signals.append({
                    "Date": dt.strftime("%Y-%m-%d"),
                    "Signal": "Basic MACD Sell",
                    "Z-Score": round(macd_zscore.iloc[i],2) if not pd.isna(macd_zscore.iloc[i]) else None
                })
    signals_df= pd.DataFrame(signals)
    if not signals_df.empty:
        signals_df["Date"]= pd.to_datetime(signals_df["Date"])
        signals_df= signals_df.sort_values("Date", ascending=False)
    return signals_df

In [28]:
# RSI and MCDX signals
def calc_rsi_entire_series(df, bullPt, bearPt):
    rsi_series = []
    last_signal = "Sell"
    for i in range(len(df)):
        if isinstance(bullPt, pd.Series) and pd.notna(bullPt.iloc[i]):
            last_signal = "Buy"
        elif isinstance(bearPt, pd.Series) and pd.notna(bearPt.iloc[i]):
            last_signal = "Sell"
        rsi_series.append(last_signal)
    return pd.Series(rsi_series, index=df.index)

def calc_mcdx_entire_series(df, upSig_MCDX, dnSig_MCDX):
    mcdx_series = []
    last_signal = "Sell"
    for i in range(len(df)):
        if pd.notna(upSig_MCDX.iloc[i]):
            last_signal = "Buy"
        elif pd.notna(dnSig_MCDX.iloc[i]):
            last_signal = "Sell"
        mcdx_series.append(last_signal)
    return pd.Series(mcdx_series, index=df.index)

In [29]:
# Weighted scoreboard
def scoreboard_for_day(
    df, i,
    rsi_series, mcdx_series,
    zero_macd_dict, basic_macd_dict,
    length_m=14,
    weight_zlma=1.0, weight_rsi=1.0, weight_mcdx=1.0, weight_dss=1.0,
    weight_zscore=1.0, weight_mg=1.0, weight_md=1.0, weight_ms=1.0,
    weight_zeromacd=1.0, weight_basicmacd=1.0
):
    # ZLMA => Buy if zlma>ema_value
    zlma_status= "Buy" if df['zlma'].iloc[i]> df['ema_value'].iloc[i] else "Sell"
    # RSI => from daily rsi_series
    rsi_status= rsi_series.iloc[i]
    # MCDX => from daily mcdx_series
    mcdx_status= mcdx_series.iloc[i]
    # DSS => Buy if DSSb>DSSsignal
    dss_status= "Buy" if df['DSSb'].iloc[i]> df['DSSsignal'].iloc[i] else "Sell"

    # Z-Score => positive => Buy
    zVal= compute_zscore(df, length_m).iloc[i]
    z_status= "Buy" if zVal>=0 else "Sell"

    # Momentum Grade
    momentum= df['close']- df['close'].shift(length_m)
    avgM= momentum.rolling(window=length_m, min_periods=length_m).mean()
    stdM= momentum.rolling(window=length_m, min_periods=length_m).std().fillna(0)
    z_m= (momentum-avgM)/stdM
    mg_z= z_m.iloc[i] if not pd.isna(z_m.iloc[i]) else 0
    if mg_z>=2:
        mg_grade="A"
    elif mg_z>=1:
        mg_grade="B"
    elif mg_z>=0:
        mg_grade="C"
    elif mg_z>=-1:
        mg_grade="D"
    elif mg_z>=-2:
        mg_grade="E"
    else:
        mg_grade="F"
    mg_status= "Buy" if mg_grade in ["A","B","C"] else "Sell"

    # Momentum Direction
    if i==0:
        md_status="Sell"
    else:
        md_status= "Buy" if momentum.iloc[i]> momentum.iloc[i-1] else "Sell"

    # Momentum State
    if i==0:
        ms_val=0
    else:
        val_now= momentum.iloc[i]
        val_prev= momentum.iloc[i-1]
        change= val_now- val_prev
        ms_val=0
        if val_now>0 and val_prev<0:
            ms_val=1
        elif val_now<0 and val_prev>0:
            ms_val=-1
        elif abs(val_now)< abs(avgM.iloc[i])*0.1:
            ms_val=0
        elif (val_now>0 and change<0) or (val_now<0 and change>0):
            ms_val=0
        elif val_now>0:
            ms_val=1
        else:
            ms_val=-1
    if ms_val>0:
        ms_status="Buy"
    elif ms_val<0:
        ms_status="Sell"
    else:
        ms_status="Neutral"

    zero_macd_status="Sell"
    if zero_macd_dict is not None:
        if zero_macd_dict["ZeroLagMACD"].iloc[i]> zero_macd_dict["signal"].iloc[i]:
            zero_macd_status="Buy"
        else:
            zero_macd_status="Sell"

    basic_macd_status="Sell"
    if basic_macd_dict is not None:
        if basic_macd_dict["basicMACD"].iloc[i]> basic_macd_dict["signal"].iloc[i]:
            basic_macd_status="Buy"
        else:
            basic_macd_status="Sell"

    scoreboard=[]
    scoreboard.append(weight_zlma if zlma_status=="Buy" else -weight_zlma)
    scoreboard.append(weight_rsi  if rsi_status=="Buy" else -weight_rsi)
    scoreboard.append(weight_mcdx if mcdx_status=="Buy" else -weight_mcdx)
    scoreboard.append(weight_dss  if dss_status=="Buy" else -weight_dss)
    scoreboard.append(weight_zscore if zVal>=0 else -weight_zscore)
    scoreboard.append(weight_mg if mg_status=="Buy" else -weight_mg)
    scoreboard.append(weight_md if md_status=="Buy" else -weight_md)

    ms_val_weighted=0
    if ms_status=="Buy":
        ms_val_weighted= weight_ms
    elif ms_status=="Sell":
        ms_val_weighted= -weight_ms
    scoreboard.append(ms_val_weighted)

    scoreboard.append(weight_zeromacd if zero_macd_status=="Buy" else -weight_zeromacd)
    scoreboard.append(weight_basicmacd if basic_macd_status=="Buy" else -weight_basicmacd)

    total_signals= sum(1 for x in scoreboard if x!=0)
    bullish_count= sum(1 for x in scoreboard if x>0)

    if total_signals==0:
        return "Neutral"
    else:
        if bullish_count>= (total_signals/2.0):
            return "Buy"
        else:
            return "Sell"

In [53]:
# Historical Trade Table
def simulate_trades_overall_signal(
    df, rsi_series, mcdx_series,
    zero_macd_dict, basic_macd_dict,
    length_m=14,
    weight_zlma=1.0, weight_rsi=1.0, weight_mcdx=1.0, weight_dss=1.0,
    weight_zscore=1.0, weight_mg=1.0, weight_md=1.0, weight_ms=1.0,
    weight_zeromacd=1.0, weight_basicmacd=1.0
):
    position = 0
    entry_price = 0.0
    trade_log = []
    cumulative_pnl_sum = 0.0
    prev_signal = "Neutral"

    for i in range(len(df)):
        day_signal = scoreboard_for_day(
            df, i, rsi_series, mcdx_series,
            zero_macd_dict, basic_macd_dict,
            length_m,
            weight_zlma, weight_rsi, weight_mcdx, weight_dss,
            weight_zscore, weight_mg, weight_md, weight_ms,
            weight_zeromacd, weight_basicmacd
        )
        price_i = df['close'].iloc[i]
        date_i = df.index[i]

        # Flip from Sell->Buy => close short
        if prev_signal in ["Sell", "Neutral"] and day_signal == "Buy":
            if position == -1:
                exit_price = price_i
                pnl_pct = ((entry_price / exit_price) - 1)*100
                cumulative_pnl_sum += pnl_pct
                trade_log.append({
                    "EntryDate": entry_dt_str,
                    "ExitDate": date_i.strftime("%Y-%m-%d"),
                    "Position": "Short",
                    # Round to 4 decimal places:
                    "EntryPrice": round(entry_price, 4),
                    "ExitPrice": round(exit_price, 4),
                    "PnL%": round(pnl_pct, 2),
                    "CumulativePnL%": round(cumulative_pnl_sum, 2)
                })
                position = 0
            position = 1
            entry_price = price_i
            entry_dt_str = date_i.strftime("%Y-%m-%d")

        # Flip from Buy->Sell => close long
        elif prev_signal in ["Buy", "Neutral"] and day_signal == "Sell":
            if position == 1:
                exit_price = price_i
                pnl_pct = ((exit_price / entry_price) - 1)*100
                cumulative_pnl_sum += pnl_pct
                trade_log.append({
                    "EntryDate": entry_dt_str,
                    "ExitDate": date_i.strftime("%Y-%m-%d"),
                    "Position": "Long",
                    "EntryPrice": round(entry_price, 4),
                    "ExitPrice": round(exit_price, 4),
                    "PnL%": round(pnl_pct, 2),
                    "CumulativePnL%": round(cumulative_pnl_sum, 2)
                })
                position = 0
            position = -1
            entry_price = price_i
            entry_dt_str = date_i.strftime("%Y-%m-%d")

        prev_signal = day_signal

    # Close any open position at the end
    if position != 0:
        final_price = df['close'].iloc[-1]
        final_date = df.index[-1]
        if position == 1:
            exit_price = final_price
            pnl_pct = ((exit_price / entry_price) - 1)*100
            cumulative_pnl_sum += pnl_pct
            trade_log.append({
                "EntryDate": entry_dt_str,
                "ExitDate": final_date.strftime("%Y-%m-%d"),
                "Position": "Long",
                "EntryPrice": round(entry_price, 4),
                "ExitPrice": round(exit_price, 4),
                "PnL%": round(pnl_pct, 2),
                "CumulativePnL%": round(cumulative_pnl_sum, 2)
            })
        else:
            exit_price = final_price
            pnl_pct = ((entry_price / exit_price) - 1)*100
            cumulative_pnl_sum += pnl_pct
            trade_log.append({
                "EntryDate": entry_dt_str,
                "ExitDate": final_date.strftime("%Y-%m-%d"),
                "Position": "Short",
                "EntryPrice": round(entry_price, 4),
                "ExitPrice": round(exit_price, 4),
                "PnL%": round(pnl_pct, 2),
                "CumulativePnL%": round(cumulative_pnl_sum, 2)
            })

    trade_df = pd.DataFrame(trade_log)
    trade_df["ExitDate_dt"] = pd.to_datetime(trade_df["ExitDate"])
    trade_df = trade_df.sort_values("ExitDate_dt", ascending=False).reset_index(drop=True)
    trade_df.drop(columns=["ExitDate_dt"], inplace=True)
    return trade_df

In [55]:
# Scoreboard with ZeroLag + Basic MACD
def extract_current_status(
    df, signalUp_ZLMA, signalDn_ZLMA, bullPt, bearPt,
    upSig_MCDX, dnSig_MCDX,
    length_m=14,
    zero_macd_dict=None, basic_macd_dict=None,

    # Weighted scoreboard
    weight_zlma=1.0, weight_rsi=1.0, weight_mcdx=1.0, weight_dss=1.0,
    weight_zscore=1.0, weight_mg=1.0, weight_md=1.0, weight_ms=1.0,
    weight_zeromacd=1.0, weight_basicmacd=1.0
):
    i= len(df)-1
    zlma_status= "Buy" if df['zlma'].iloc[i]> df['ema_value'].iloc[i] else "Sell"

    # RSI day-by-day
    daily_rsi_series  = calc_rsi_entire_series(df, bullPt, bearPt)
    rsi_status        = daily_rsi_series.iloc[i]
    # MCDX day-by-day
    daily_mcdx_series = calc_mcdx_entire_series(df, upSig_MCDX, dnSig_MCDX)
    mcdx_status       = daily_mcdx_series.iloc[i]
    # DSS => buy if DSSb>DSSsignal
    dss_status= "Buy" if df['DSSb'].iloc[i]> df['DSSsignal'].iloc[i] else "Sell"

    # Z-Score
    zVal= compute_zscore(df, length_m).iloc[i]
    z_status= "Buy" if zVal>=0 else "Sell"

    # Momentum
    momentum= df['close']- df['close'].shift(length_m)
    avgM= momentum.rolling(window=length_m, min_periods=length_m).mean()
    stdM= momentum.rolling(window=length_m, min_periods=length_m).std().fillna(0)
    z_m= (momentum-avgM)/ stdM
    mg_z= z_m.iloc[i] if not pd.isna(z_m.iloc[i]) else 0
    if mg_z>=2:
        mg_letter="A"
    elif mg_z>=1:
        mg_letter="B"
    elif mg_z>=0:
        mg_letter="C"
    elif mg_z>=-1:
        mg_letter="D"
    elif mg_z>=-2:
        mg_letter="E"
    else:
        mg_letter="F"
    mg_status= "Buy" if mg_letter in ["A","B","C"] else "Sell"

    # momentum direction
    if i==0:
        md_status="Sell"
    else:
        md_status= "Buy" if momentum.iloc[i]> momentum.iloc[i-1] else "Sell"

    # momentum state
    if i==0:
        ms_val=0
    else:
        val_now= momentum.iloc[i]
        val_prev= momentum.iloc[i-1]
        change= val_now- val_prev
        ms_val=0
        if val_now>0 and val_prev<0:
            ms_val=1
        elif val_now<0 and val_prev>0:
            ms_val=-1
        elif abs(val_now)< abs(avgM.iloc[i])*0.1:
            ms_val=0
        elif (val_now>0 and change<0) or (val_now<0 and change>0):
            ms_val=0
        elif val_now>0:
            ms_val=1
        else:
            ms_val=-1
    if ms_val>0:
        ms_status="Buy"
    elif ms_val<0:
        ms_status="Sell"
    else:
        ms_status="Neutral"

    zero_macd_status= "Sell"
    if zero_macd_dict is not None:
        if zero_macd_dict["ZeroLagMACD"].iloc[i]> zero_macd_dict["signal"].iloc[i]:
            zero_macd_status= "Buy"
        else:
            zero_macd_status= "Sell"

    basic_macd_status="Sell"
    if basic_macd_dict is not None:
        if basic_macd_dict["basicMACD"].iloc[i]> basic_macd_dict["signal"].iloc[i]:
            basic_macd_status= "Buy"
        else:
            basic_macd_status= "Sell"

    scoreboard=[]
    scoreboard.append(weight_zlma if zlma_status=="Buy" else -weight_zlma)
    scoreboard.append(weight_rsi  if rsi_status=="Buy" else -weight_rsi)
    scoreboard.append(weight_mcdx if mcdx_status=="Buy" else -weight_mcdx)
    scoreboard.append(weight_dss  if dss_status=="Buy" else -weight_dss)
    scoreboard.append(weight_zscore if zVal>=0 else -weight_zscore)
    scoreboard.append(weight_mg if mg_status=="Buy" else -weight_mg)
    scoreboard.append(weight_md if md_status=="Buy" else -weight_md)

    ms_val_weighted=0
    if ms_status=="Buy":
        ms_val_weighted= weight_ms
    elif ms_status=="Sell":
        ms_val_weighted= -weight_ms
    scoreboard.append(ms_val_weighted)

    scoreboard.append(weight_zeromacd if zero_macd_status=="Buy" else -weight_zeromacd)
    scoreboard.append(weight_basicmacd if basic_macd_status=="Buy" else -weight_basicmacd)

    total_signals= sum(1 for x in scoreboard if x!=0)
    bullish_count= sum(1 for x in scoreboard if x>0)
    total_score= sum(scoreboard)

    if total_signals==0:
        overall= "Neutral"
    else:
        if bullish_count>= (total_signals/2.0):
            overall= "Buy"
        else:
            overall= "Sell"

    indicators= [
        "ZLMA","RSI","MCDX","DSS","Z-Score",
        "Momentum Grade","Momentum Direction","Momentum State",
        "ZeroLag MACD","Basic MACD","ScoreSum","Overall Trade"
    ]
    signals_=[
        zlma_status,
        rsi_status,
        mcdx_status,
        dss_status,
        round(zVal,2),
        mg_letter,
        ("Increasing" if md_status=="Buy" else "Decreasing"),
        ms_status,
        zero_macd_status,
        basic_macd_status,
        str(round(total_score,2)),
        overall
    ]
    return pd.DataFrame({"Indicator": indicators, "Current Signal": signals_})

In [32]:
# Plot (ZLMA, Bressert, MCDX, MCDX Bars, ZeroLag MACD, IV, Momentum Z-Score, Basic MACD)
def create_generic_plot(df, ticker, start_date, end_date,
                        ema_value, zlma, signalUp_ZLMA, signalDn_ZLMA, zlma_color, ema_color,
                        rsi_ma_base, rsi_upper_bound, rsi_lower_bound, bullPt, bearPt,
                        b_X, b_DSSb, b_DSSsignal,
                        hbma, threshold, upSig_MCDX, dnSig_MCDX,
                        Dump, DnCandle, PumpCandle, Retest, Banker,
                        iv_series, zero_macd_dict, basic_macd_dict=None,
                        momentum_length=14):
    fig, axs = plt.subplots(8,1,sharex=True,figsize=(12,20),
                            gridspec_kw={"height_ratios":[2,1,1,1,1,1,1,1]})
    fig.suptitle(f"{ticker} - Generic Multi-Panel Chart with Momentum & Dual MACD", fontsize=14)
    x_vals = mdates.date2num(df.index.to_pydatetime())

    # Panel 1: Price + ZLMA + RSI + Momentum text
    for i in range(len(df)):
        o, c, h, l = df['open'].iloc[i], df['close'].iloc[i], df['high'].iloc[i], df['low'].iloc[i]
        color= 'green' if c>= o else 'red'
        axs[0].plot([x_vals[i], x_vals[i]], [l,h], color=color, linewidth=1, zorder=1)
        candle_width= 0.6
        axs[0].add_patch(Rectangle((x_vals[i]-candle_width/2, o), candle_width, c-o,
                                   facecolor=color, edgecolor=color, zorder=2))
    axs[0].plot(df.index, df['EMA_50'],  label="EMA 50",  color='blue',   linewidth=1.5, zorder=3)
    axs[0].plot(df.index, df['EMA_100'], label="EMA 100", color='orange', linewidth=1.5, zorder=3)
    axs[0].plot(df.index, df['EMA_200'], label="EMA 200", color='purple', linewidth=1.5, zorder=3)
    axs[0].plot(df.index, df['EMA_500'], label="EMA 500", color='brown',  linewidth=1.5, zorder=3)
    axs[0].plot(df.index, ema_value, label="EMA (Trend)", color=ema_color, linewidth=2, zorder=4)
    axs[0].plot(df.index, zlma,      label="ZLMA",        color=zlma_color, linewidth=2, zorder=4)
    axs[0].fill_between(df.index, zlma, ema_value, where=(zlma>=ema_value), facecolor="darkgreen", alpha=0.3, interpolate=True, zorder=3)
    axs[0].fill_between(df.index, zlma, ema_value, where=(zlma<ema_value), facecolor="darkred", alpha=0.3, interpolate=True, zorder=3)
    axs[0].scatter(df.index, zlma.where(signalUp_ZLMA), color="cyan",    marker="o", s=50, label="ZLMA Buy",  zorder=5)
    axs[0].scatter(df.index, zlma.where(signalDn_ZLMA), color="magenta", marker="o", s=50, label="ZLMA Sell", zorder=5)

    axs[0].plot(df.index, rsi_ma_base,     label="RSI Trail Base",  color="gray", linestyle="--", linewidth=1)
    axs[0].plot(df.index, rsi_upper_bound, label="RSI Trail Upper", color="blue", linewidth=1)
    axs[0].plot(df.index, rsi_lower_bound, label="RSI Trail Lower", color="red",  linewidth=1)
    if isinstance(bullPt, pd.Series):
        axs[0].scatter(df.index, bullPt, color="cyan",    marker="^", s=50, label="RSI Buy",  zorder=6)
    if isinstance(bearPt, pd.Series):
        axs[0].scatter(df.index, bearPt, color="magenta", marker="v", s=50, label="RSI Sell", zorder=6)
    axs[0].fill_between(df.index, rsi_ma_base,     rsi_upper_bound, facecolor="darkgreen", alpha=0.2, interpolate=True)
    axs[0].fill_between(df.index, rsi_lower_bound, rsi_ma_base,     facecolor="darkred",   alpha=0.2, interpolate=True)

    fomc_dates= get_fomc_dates(start_date, end_date)
    for i, dt in enumerate(fomc_dates):
        axs[0].axvline(dt, color="purple", linestyle="--", linewidth=1, label="FOMC" if i==0 else "")

    axs[0].set_ylabel("Price")
    # Legend bottom-right
    axs[0].legend(loc="lower right", ncol=3, fontsize=8)

    # Momentum text
    momentum= df['close']- df['close'].shift(momentum_length)
    avgM= momentum.rolling(window=momentum_length, min_periods=momentum_length).mean()
    stdM= momentum.rolling(window=momentum_length, min_periods=momentum_length).std().fillna(0)
    zScore_m= (momentum- avgM)/ stdM
    if len(zScore_m.dropna())>0:
        last_z= zScore_m.iloc[-1]
    else:
        last_z= np.nan
    if not np.isnan(last_z):
        if last_z>=2:
            gradeStr= "A (Strong Positive Momentum)"
            gradeColor= "green"
        elif last_z>=1:
            gradeStr= "B (Moderate Positive Momentum)"
            gradeColor= "lightgreen"
        elif last_z>=0:
            gradeStr= "C (Weak Positive Momentum)"
            gradeColor= "goldenrod"
        elif last_z>=-1:
            gradeStr= "D (Weak Negative Momentum)"
            gradeColor= "orange"
        elif last_z>=-2:
            gradeStr= "E (Moderate Negative Momentum)"
            gradeColor= "red"
        else:
            gradeStr= "F (Strong Negative Momentum)"
            gradeColor= "darkred"
    else:
        gradeStr= "N/A"
        gradeColor= "white"
    if len(df)>1:
        directionIncreasing= (momentum.iloc[-1]> momentum.iloc[-2])
        dirStr= "Increasing" if directionIncreasing else "Decreasing"
        dirColor= "green" if directionIncreasing else "red"
        change= momentum.iloc[-1]- momentum.iloc[-2]
        if momentum.iloc[-1]* momentum.iloc[-2]<0:
            stateStr= "Turning Up" if momentum.iloc[-1]>0 else "Turning Down"
            stateColor= "orange"
        elif abs(momentum.iloc[-1])< abs(avgM.iloc[-1])*0.1:
            stateStr= "Consolidating"
            stateColor= "yellow"
        elif (momentum.iloc[-1]>0 and change<0) or (momentum.iloc[-1]<0 and change>0):
            stateStr= "Stalling"
            stateColor= "lightgray"
        elif momentum.iloc[-1]>0:
            stateStr= "Positive Trending"
            stateColor= "green"
        else:
            stateStr= "Negative Trending"
            stateColor= "red"
    else:
        dirStr, dirColor= "N/A","white"
        stateStr, stateColor= "N/A","white"

    axs[0].text(0.01,0.95, f"Momentum Grade: {gradeStr} (Z-Score: {last_z:.2f})",
                transform= axs[0].transAxes, fontsize=10, color= gradeColor,
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))
    axs[0].text(0.01,0.90, f"Momentum Direction: {dirStr}",
                transform= axs[0].transAxes, fontsize=10, color= dirColor,
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))
    axs[0].text(0.01,0.85, f"Momentum State: {stateStr}",
                transform= axs[0].transAxes, fontsize=10, color= stateColor,
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

    # Panel 2: Bressert
    axs[1].plot(df.index, b_X, label="X (EMA of Y)", color="black", linewidth=2)
    marker_colors= ['black']+ ['red' if b_X.iloc[i]< b_X.iloc[i-1] else 'green' for i in range(1,len(b_X))]
    axs[1].scatter(df.index, b_X, c=marker_colors, s=20)
    axs[1].plot(df.index, b_DSSb,       label="DSSb",       color="blue",    linewidth=2)
    axs[1].plot(df.index, b_DSSsignal,  label="DSSsignal",  color="magenta", linewidth=2)
    axs[1].axhline(50, color="gray", linewidth=1)
    axs[1].axhline(80, color="red",  linewidth=2)
    axs[1].axhline(20, color="green",linewidth=2)
    axs[1].set_ylabel("Bressert")
    axs[1].legend(loc="lower left", fontsize=8)

    # Panel 3: MCDX HBMA & Signals
    axs[2].plot(df.index, hbma, label="HBMA", color="black", linewidth=2, zorder=3)
    axs[2].axhline(threshold, color="gray", linestyle="--", label="Threshold", zorder=2)
    axs[2].scatter(df.index, upSig_MCDX, color="green", marker="o", s=50, label="MCDX Buy",  zorder=4)
    axs[2].scatter(df.index, dnSig_MCDX, color="red",   marker="o", s=50, label="MCDX Sell", zorder=4)
    axs[2].set_ylabel("MCDX HBMA")
    axs[2].legend(loc="lower left", fontsize=8)

    # Panel 4: MCDX Bars
    axs[3].bar(df.index, Dump,       width=0.8, color="red",      alpha=0.7, label="Dump",        zorder=1)
    axs[3].bar(df.index, DnCandle,   width=0.8, color="darkgray", alpha=0.7, label="Down Candle", zorder=1)
    axs[3].bar(df.index, PumpCandle, width=0.8, color="green",    alpha=0.7, label="Pump Candle", zorder=1)
    axs[3].bar(df.index, Retest,     width=0.8, color="darkred",  alpha=0.7, label="Retest",      zorder=1)
    axs[3].bar(df.index, Banker,     width=0.8, color="#84AFC9",  alpha=0.7, label="Banker",      zorder=1)
    axs[3].set_ylabel("MCDX Bars")
    axs[3].legend(loc="lower left", fontsize=8)

    # Panel 5: Zero Lag MACD
    zmacd= zero_macd_dict
    axs[4].fill_between(df.index, zmacd["ZeroLagMACD"], zmacd["signal"],
                        where=(zmacd["ZeroLagMACD"]>= zmacd["signal"]),
                        facecolor="green", alpha=0.3, interpolate=True)
    axs[4].fill_between(df.index, zmacd["ZeroLagMACD"], zmacd["signal"],
                        where=(zmacd["ZeroLagMACD"]< zmacd["signal"]),
                        facecolor="red", alpha=0.3, interpolate=True)
    axs[4].plot(df.index, zmacd["ZeroLagMACD"], label="ZeroLag MACD", color="green", linewidth=1)
    axs[4].plot(df.index, zmacd["signal"],      label="ZMACD Signal", color="red",   linewidth=1)
    axs[4].bar(df.index, zmacd["upHist"]*2,   label="ZMACD Hist Up",   color="gray", width=0.8)
    axs[4].bar(df.index, zmacd["downHist"]*2, label="ZMACD Hist Down", color="red",  width=0.8)
    axs[4].scatter(df.index, zmacd["dotUP"], color="green", marker="o", s=50, label="ZMACD Dot Up")
    axs[4].scatter(df.index, zmacd["dotDN"], color="red",   marker="o", s=50, label="ZMACD Dot Down")
    axs[4].set_ylabel("ZeroLag MACD")
    axs[4].legend(loc="lower left", fontsize=8)

    # Panel 6: VIX
    axs[5].plot(df.index, iv_series, label="VIX", color="darkorange", linewidth=2)
    axs[5].set_ylabel("IV")
    axs[5].legend(loc="lower left", fontsize=8)
    axs[5].xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
    for tick in axs[5].get_xticklabels():
        tick.set_rotation(45)

    # Panel 7: Momentum Z-Score
    zScore_panel= compute_zscore(df, momentum_length)
    axs[6].axhline(0, color="gray", linestyle="--")
    axs[6].fill_between(df.index, zScore_panel, 0, where=(zScore_panel>0), facecolor="green", alpha=0.3)
    axs[6].fill_between(df.index, zScore_panel, 0, where=(zScore_panel<0), facecolor="red", alpha=0.3)
    axs[6].plot(df.index, zScore_panel, label="Momentum Z-Score", color="black", linewidth=1.5)
    axs[6].set_ylabel("Momentum")
    axs[6].legend(loc="lower left", fontsize=8)

    # Panel 8: Basic MACD
    if basic_macd_dict:
        macd_line= basic_macd_dict["basicMACD"]
        sig_line=  basic_macd_dict["signal"]
        hist_line= basic_macd_dict["hist"]
        axs[7].fill_between(df.index, macd_line, sig_line,
                            where=(macd_line>= sig_line),
                            facecolor="green", alpha=0.3, interpolate=True)
        axs[7].fill_between(df.index, macd_line, sig_line,
                            where=(macd_line< sig_line),
                            facecolor="red", alpha=0.3, interpolate=True)
        axs[7].plot(df.index, macd_line, label="Basic MACD", color="green", linewidth=1)
        axs[7].plot(df.index, sig_line,  label="Basic MACD Signal", color="red", linewidth=1)
        axs[7].bar(df.index, hist_line*2, label="Basic MACD Hist", color="gray", width=0.8)
        axs[7].set_ylabel("Basic MACD")
        axs[7].legend(loc="lower left", fontsize=8)
    else:
        axs[7].text(0.5,0.5, "No Basic MACD Data", ha="center", va="center", transform=axs[7].transAxes)
        axs[7].set_ylabel("Basic MACD")

    plt.tight_layout()
    return fig

def figure_to_pil(fig):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight")
    buf.seek(0)
    return Image.open(buf)

In [33]:
# Main Chart-Generating Function with trade log
def generate_plot(
    ticker="SPY",
    start_date="2022-01-01",
    end_date="2023-01-01",

    # Weighted scoreboard
    weight_zlma=1.0,
    weight_rsi=1.0,
    weight_mcdx=1.0,
    weight_dss=1.0,
    weight_zscore=1.0,
    weight_mg=1.0,
    weight_md=1.0,
    weight_ms=1.0,

    # Two MACD weights
    weight_zeromacd=1.0,
    weight_basicmacd=1.0
):
    try:
        df = download_data(ticker, start_date, end_date)
        if df.empty:
            raise gr.Error(f"No data for {ticker} from {start_date} to {end_date}")
        for col in ["open","high","low","close","volume"]:
            if col not in df.columns:
                raise gr.Error(f"Missing {col} data for {ticker}")

        # Basic EMAs
        df['EMA_50'] = exp_average(df['close'], 50)
        df['EMA_100']= exp_average(df['close'], 100)
        df['EMA_200']= exp_average(df['close'], 200)
        df['EMA_500']= exp_average(df['close'], 500)

        # ZLMA
        movAvgLength= 15
        ema_value= exp_average(df['close'], movAvgLength)
        df['ema_value']= ema_value
        correction= df['close']+(df['close']-ema_value)
        zlma= exp_average(correction, movAvgLength)
        df['zlma']= zlma

        signalUp_ZLMA= (zlma>ema_value)&(zlma.shift(1)<= ema_value.shift(1))
        signalDn_ZLMA= (zlma<ema_value)&(zlma.shift(1)>= ema_value.shift(1))
        zlma_color= "green" if zlma.iloc[-1]> zlma.iloc[-2] else "red"
        ema_color=  "green" if ema_value.iloc[-1]< zlma.iloc[-1] else "red"

        # Bressert
        df= compute_bressert(df,8,13)
        b_X= df['X']
        b_DSSb= df['DSSb']
        b_DSSsignal= df['DSSsignal']

        # MCDX + RSI
        RSIBaseBanker= 50;   RSIPeriodBanker= 50
        RSIBaseHotMoney=30; RSIPeriodHotMoney= 40
        SensitivityBanker=1.5; SensitivityHotMoney=0.7
        threshold= 8.5

        rsi_Banker= rsi_function(df['close'], SensitivityBanker, RSIPeriodBanker, RSIBaseBanker)
        rsi_HotMoney= rsi_function(df['close'], SensitivityHotMoney, RSIPeriodHotMoney, RSIBaseHotMoney)
        hot= rsi_HotMoney
        bank= rsi_Banker
        hotma2= wilder_average(hot,2)
        hotma7= wilder_average(hot,7)
        hotma31= wilder_average(hot,31)
        hotma= exp_average((hotma2*34 + hotma7*33 + hotma31*33)/100,2)
        bankma2= df['close'].rolling(window=2, min_periods=2).mean()
        bankma7= exp_average(bank,7)
        bankma31=exp_average(bank,31)
        bankma= ((bankma2*70 + bankma7*20 + bankma31*10)/100).rolling(window=2, min_periods=2).mean()
        banksignal= wilder_average(bankma,4)
        df['banksignal']= banksignal

        hbAvg= ((hot*10)+(hotma*35)+(wilder_average(hotma,2)*15)+(bankma*35)+(banksignal*5))/100
        hbma= vwma(hbAvg,2, df['volume'])
        df['hbma']= hbma

        downtrendsignal= (hotma.shift(1)>= wilder_average(hotma,2).shift(1))&(hotma< wilder_average(hotma,2))
        uptrendsignal=   (hotma.shift(1)<= wilder_average(hotma,2).shift(1))&(hotma> wilder_average(hotma,2))
        upSig_MCDX= hbma.where(uptrendsignal,   np.nan)
        dnSig_MCDX= hbma.where(downtrendsignal, np.nan)

        Dump= bank.where(bank< bank.shift(1)/1.75, np.nan)
        dnCond= (bank<bank.shift(1))&(bank<bank.shift(2))&(bank.shift(1)<bank.shift(2))& \
                (bank<bank.shift(3))&(bank<bank.shift(4))&(bank.shift(3)<bank.shift(4))& \
                (bank.shift(6)>8.5)&(bank<10)
        DnCandle=   bank.where(dnCond, np.nan)
        PumpCandle= bank.where(bank> hbma, np.nan)
        Retest=     bank.where((banksignal> bankma)&(bank>0), np.nan)
        Banker=     bank

        # RSI trail
        lookbackPeriod=15
        atrLength=27
        atrMultiplier=0.1
        rsiLowerThreshold=40
        rsiUpperThreshold=60
        ohlc4= (df['open']+df['high']+df['low']+df['close'])/4
        rsi_ma_base= t3(ohlc4, length=lookbackPeriod, vf=0.7)
        df['rsi_ma_base']= rsi_ma_base
        tr_series= pd.concat([
            df['high']-df['low'],
            abs(df['high']-df['close'].shift(1)),
            abs(df['low']-df['close'].shift(1))
        ], axis=1).max(axis=1)
        nzTR= tr_series.fillna(df['high']-df['low'])
        f_volatility= wilder_average(nzTR,atrLength)* atrMultiplier
        rsi_upper_bound= rsi_ma_base+ ((rsiUpperThreshold-50)/10)* f_volatility
        rsi_lower_bound= rsi_ma_base- ((50-rsiLowerThreshold)/10)* f_volatility

        crossUp= (ohlc4> rsi_upper_bound)&(ohlc4.shift(1)<= rsi_upper_bound.shift(1))
        crossDn= (df['close']<rsi_lower_bound)&(df['close'].shift(1)>= rsi_lower_bound.shift(1))
        bullPt=   rsi_lower_bound.where(crossUp, np.nan)
        bearPt=   rsi_upper_bound.where(crossDn, np.nan)

        # VIX => iv_series
        vix_df= yf.download("^VIX", start=pd.to_datetime(start_date), end=pd.to_datetime(end_date))
        if vix_df.empty:
            iv_series= pd.Series(np.nan, index=df.index)
        else:
            vix_df.index= pd.to_datetime(vix_df.index)
            iv_series= vix_df["Close"].reindex(df.index, method="ffill")

        # ZeroLag MACD
        zero_macd_dict= compute_zero_lag_macd(df['close'], fastLength=12, slowLength=26, signalLength=9,
                                              MacdEmaLength=9, useEma=True, useOldAlgo=False)

        # Basic MACD
        # basic_macd_dict= compute_basic_macd(df['close'], fast=12, slow=26, signal=9)
        basic_macd_dict= compute_basic_macd(df['close'], fast=30, slow=50, signal=9)

        # Create multi-panel figure
        fig= create_generic_plot(
            df, ticker, start_date, end_date,
            ema_value, zlma, signalUp_ZLMA, signalDn_ZLMA, zlma_color, ema_color,
            rsi_ma_base, rsi_upper_bound, rsi_lower_bound, bullPt, bearPt,
            b_X, b_DSSb, b_DSSsignal,
            hbma, threshold, upSig_MCDX, dnSig_MCDX,
            Dump, DnCandle, PumpCandle, Retest, Banker,
            iv_series, zero_macd_dict, basic_macd_dict,
            momentum_length=14
        )

        # Build historical signals
        signals_zmacd_df= extract_zero_macd_signals(df, zero_macd_dict, length_m=14)
        signals_bmacd_df= extract_basic_macd_signals(df, basic_macd_dict, length_m=14)
        signals_others_df= extract_signals(df, signalUp_ZLMA, signalDn_ZLMA, bullPt, bearPt,
                                           upSig_MCDX, dnSig_MCDX, length_m=14)
        momentum_signals_df= extract_momentum_signals(df, length_m=14)
        historical_signals_df= pd.concat([
            signals_others_df,
            momentum_signals_df,
            signals_zmacd_df,
            signals_bmacd_df
        ], ignore_index=True)
        if not historical_signals_df.empty:
            historical_signals_df["Date"]= pd.to_datetime(historical_signals_df["Date"])
            historical_signals_df= historical_signals_df.sort_values("Date", ascending=False)

        # Current scoreboard
        current_status_df= extract_current_status(
            df, signalUp_ZLMA, signalDn_ZLMA, bullPt, bearPt,
            upSig_MCDX, dnSig_MCDX,
            length_m=14,
            zero_macd_dict= zero_macd_dict,
            basic_macd_dict= basic_macd_dict,
            weight_zlma= weight_zlma,
            weight_rsi= weight_rsi,
            weight_mcdx= weight_mcdx,
            weight_dss= weight_dss,
            weight_zscore= weight_zscore,
            weight_mg= weight_mg,
            weight_md= weight_md,
            weight_ms= weight_ms,
            weight_zeromacd= weight_zeromacd,
            weight_basicmacd= weight_basicmacd
        )

        # Day-by-day scoreboard => trade log
        daily_rsi_series= calc_rsi_entire_series(df, bullPt, bearPt)
        daily_mcdx_series= calc_mcdx_entire_series(df, upSig_MCDX, dnSig_MCDX)
        trade_log_df= simulate_trades_overall_signal(
            df, daily_rsi_series, daily_mcdx_series,
            zero_macd_dict, basic_macd_dict,
            length_m=14,
            weight_zlma= weight_zlma,
            weight_rsi= weight_rsi,
            weight_mcdx= weight_mcdx,
            weight_dss= weight_dss,
            weight_zscore= weight_zscore,
            weight_mg= weight_mg,
            weight_md= weight_md,
            weight_ms= weight_ms,
            weight_zeromacd= weight_zeromacd,
            weight_basicmacd= weight_basicmacd
        )

        buf= io.BytesIO()
        fig.savefig(buf, format="png", bbox_inches="tight")
        buf.seek(0)
        pil_img= Image.open(buf)
        plt.close(fig)

        return pil_img, current_status_df, historical_signals_df, trade_log_df
    except Exception as e:
        debug_print(f"Error: {e}")
        raise gr.Error(f"An error occurred: {e}")

In [34]:
# Save Historical Data
def save_historical_data(ticker="SPY", start_date="2022-01-01", end_date="2023-01-01",
                         data_filename="full_data.csv", signals_filename="signals_data.csv"):
    try:
        df= download_data(ticker, start_date, end_date)
        if df.empty:
            raise gr.Error(f"No data for {ticker} from {start_date} to {end_date}")
        for col in ["open","high","low","close","volume"]:
            if col not in df.columns:
                raise gr.Error(f"Missing {col} data for {ticker}")

        # Basic EMAs
        df['EMA_50']  = exp_average(df['close'], 50)
        df['EMA_100'] = exp_average(df['close'], 100)
        df['EMA_200'] = exp_average(df['close'], 200)
        df['EMA_500'] = exp_average(df['close'], 500)

        # ZLMA
        movAvgLength= 15
        ema_value= exp_average(df['close'], movAvgLength)
        df['ema_value']= ema_value
        correction= df['close']+ (df['close']- df['ema_value'])
        df['zlma']= exp_average(correction,movAvgLength)
        signalUp_ZLMA= (df['zlma']> df['ema_value'])&(df['zlma'].shift(1)<= df['ema_value'].shift(1))
        signalDn_ZLMA= (df['zlma']< df['ema_value'])&(df['zlma'].shift(1)>= df['ema_value'].shift(1))

        # Bressert
        df= compute_bressert(df,8,13)

        # MCDX
        RSIBaseBanker=50; RSIPeriodBanker=50
        RSIBaseHotMoney=30; RSIPeriodHotMoney=40
        SensitivityBanker=1.5; SensitivityHotMoney=0.7
        rsi_Banker= rsi_function(df['close'], SensitivityBanker,   RSIPeriodBanker,    RSIBaseBanker)
        rsi_HotMoney= rsi_function(df['close'], SensitivityHotMoney, RSIPeriodHotMoney, RSIBaseHotMoney)
        hot= rsi_HotMoney
        bank= rsi_Banker
        hotma2= wilder_average(hot,2)
        hotma7= wilder_average(hot,7)
        hotma31= wilder_average(hot,31)
        hotma= exp_average((hotma2*34 + hotma7*33 + hotma31*33)/100,2)

        bankma2= df['close'].rolling(window=2, min_periods=2).mean()
        bankma7= exp_average(bank,7)
        bankma31= exp_average(bank,31)
        bankma= ((bankma2*70 + bankma7*20 + bankma31*10)/100).rolling(window=2,min_periods=2).mean()
        banksignal= wilder_average(bankma,4)
        df['banksignal']= banksignal

        hbAvg= ((hot*10)+(hotma*35)+(wilder_average(hotma,2)*15)+(bankma*35)+(banksignal*5))/100
        hbma= vwma(hbAvg,2,df['volume'])
        df['hbma']= hbma

        uptrendsignal= (hotma.shift(1)<= wilder_average(hotma,2).shift(1))&(hotma> wilder_average(hotma,2))
        downtrendsignal=(hotma.shift(1)>= wilder_average(hotma,2).shift(1))&(hotma< wilder_average(hotma,2))
        upSig_MCDX= df['hbma'].where(uptrendsignal,   np.nan)
        dnSig_MCDX= df['hbma'].where(downtrendsignal, np.nan)

        lookbackPeriod= 15
        atrLength= 27
        atrMultiplier= 0.1
        rsiLowerThreshold=40
        rsiUpperThreshold=60
        ohlc4= (df['open']+df['high']+df['low']+df['close'])/4
        rsi_ma_base= t3(ohlc4, length=lookbackPeriod, vf=0.7)
        df['rsi_ma_base']= rsi_ma_base
        tr_series= pd.concat([
            df['high']-df['low'],
            abs(df['high']-df['close'].shift(1)),
            abs(df['low']-df['close'].shift(1))
        ], axis=1).max(axis=1)
        nzTR= tr_series.fillna(df['high']-df['low'])
        f_volatility= wilder_average(nzTR, atrLength)*atrMultiplier
        rsi_upper_bound= rsi_ma_base+ ((rsiUpperThreshold-50)/10)* f_volatility
        rsi_lower_bound= rsi_ma_base- ((50-rsiLowerThreshold)/10)* f_volatility
        crossUp= (ohlc4> rsi_upper_bound)&(ohlc4.shift(1)<= rsi_upper_bound.shift(1))
        crossDn= (df['close']< rsi_lower_bound)&(df['close'].shift(1)>= rsi_lower_bound.shift(1))
        bullPt= rsi_lower_bound.where(crossUp, np.nan)
        bearPt= rsi_upper_bound.where(crossDn, np.nan)

        from copy import deepcopy
        # For ZeroLag
        zero_macd_dict= compute_zero_lag_macd(df['close'], 12, 26, 9, 9, True,False)

        # We'll not store Basic MACD columns in CSV for now
        # signals
        signals_others_df= extract_signals(df, signalUp_ZLMA, signalDn_ZLMA, bullPt, bearPt, upSig_MCDX, dnSig_MCDX, length_m=14)
        momentum_signals_df= extract_momentum_signals(df, length_m=14)
        zero_macd_signals_df= extract_zero_macd_signals(df, zero_macd_dict, length_m=14)
        historical_signals_df= pd.concat([
            signals_others_df,
            momentum_signals_df,
            zero_macd_signals_df
        ], ignore_index=True)

        if not historical_signals_df.empty:
            historical_signals_df["Date"]= pd.to_datetime(historical_signals_df["Date"])
            historical_signals_df= historical_signals_df.sort_values("Date", ascending=False)

        # 0/1 columns
        indicator_list= ["ZLMA","RSI","MCDX","DSS","ZeroLag MACD"]
        signals_grouped= historical_signals_df.groupby("Date")["Signal"].apply(list)
        for ind in indicator_list:
            df[ind+"_Buy"]= df.index.to_series().apply(
                lambda d: 1 if any((ind in s and "Buy" in s) for s in signals_grouped.get(d,[])) else 0
            )
            df[ind+"_Sell"]= df.index.to_series().apply(
                lambda d: 1 if any((ind in s and "Sell" in s) for s in signals_grouped.get(d,[])) else 0
            )

        df.to_csv(data_filename)
        historical_signals_df.to_csv(signals_filename, index=False)
        return f"Saved {data_filename} and {signals_filename} successfully."

    except Exception as e:
        debug_print(f"Error: {e}")
        raise gr.Error(f"An error occurred: {e}")

In [35]:
# DQN + environment
class TradingEnv:
    def __init__(self, df, window_size=3, max_hold_days=10):
        self.df = df.reset_index(drop=True).copy()
        self.length = len(self.df)
        self.window_size = window_size
        self.max_hold_days = max_hold_days

        self.signal_cols = [
            "ZLMA_Buy","ZLMA_Sell","RSI_Buy","RSI_Sell",
            "MCDX_Buy","MCDX_Sell","DSS_Buy","DSS_Sell",
            "ZeroLag MACD_Buy","ZeroLag MACD_Sell"
        ]
        for col in self.signal_cols:
            if col not in self.df.columns:
                self.df[col] = 0

        self.current_step = 0
        self.position = 0
        self.hold_counter = 0
        self.history_buffer = []

    def reset(self):
        self.current_step = 0
        self.position = 0
        self.hold_counter = 0
        self.history_buffer = []
        for _ in range(self.window_size):
            self._append_history()
        return self._get_state()

    def step(self, action):
        """
        0=Hold,1=Go Long,2=Go Short
        """
        prev_position= self.position
        current_price= self.df.loc[self.current_step, 'close']
        reward= 0.0
        if self.current_step>0:
            price_prev= self.df.loc[self.current_step-1, 'close']
            reward= (current_price- price_prev)* prev_position
        if (prev_position==1 and action==2) or (prev_position==-1 and action==1):
            reward-= 0.001* current_price

        new_position= self.position
        if action==1:
            new_position=1
        elif action==2:
            new_position=-1

        if new_position!= prev_position:
            self.position= new_position
            if self.position==0:
                self.hold_counter=0
            else:
                self.hold_counter=1
        else:
            if self.position!=0:
                self.hold_counter+=1

        if self.hold_counter> self.max_hold_days:
            reward-= 0.02* current_price
            self.position=0
            self.hold_counter=0

        day_signals= self.df.loc[self.current_step, self.signal_cols].values
        buy_sum= day_signals[0::2].sum()
        sell_sum= day_signals[1::2].sum()
        if self.position==1:
            reward+= 0.05* buy_sum
            reward-= 0.02* sell_sum
        elif self.position==-1:
            reward+= 0.05* sell_sum
            reward-= 0.02* buy_sum

        self.current_step+=1
        done= (self.current_step>= self.length-1)
        self._append_history()
        return self._get_state(), reward, done

    def _append_history(self):
        if self.current_step< self.length:
            row= self.df.loc[self.current_step, ["close"]+ self.signal_cols].values
        else:
            row= self.df.loc[self.length-1, ["close"]+ self.signal_cols].values
        self.history_buffer.append(row.astype(np.float32))
        if len(self.history_buffer)> self.window_size:
            self.history_buffer.pop(0)

    def _get_state(self):
        arr= np.array(self.history_buffer)
        flattened= arr.flatten()
        return np.concatenate([flattened, [float(self.position)]], dtype=np.float32)

class QNetwork(nn.Module):
    def __init__(self, input_dim=34, hidden_dim=64, output_dim=3):
        super(QNetwork,self).__init__()
        self.fc1= nn.Linear(input_dim, hidden_dim)
        self.fc2= nn.Linear(hidden_dim, hidden_dim)
        self.fc3= nn.Linear(hidden_dim, output_dim)
    def forward(self,x):
        x= F.relu(self.fc1(x))
        x= F.relu(self.fc2(x))
        return self.fc3(x)

def train_dqn_from_df(df, episodes=5, gamma=0.99, lr=1e-3, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01):
    env= TradingEnv(df, window_size=3, max_hold_days=10)
    policy_net= QNetwork(input_dim=34, hidden_dim=64, output_dim=3)
    optimizer= optim.Adam(policy_net.parameters(), lr=lr)

    rewards_per_episode= []
    for ep in range(episodes):
        state= env.reset()
        done=False
        ep_reward=0.0
        while not done:
            if np.random.rand()<epsilon:
                action= np.random.randint(0,3)
            else:
                with torch.no_grad():
                    s= torch.FloatTensor(state).unsqueeze(0)
                    q_values= policy_net(s)
                    action= q_values.argmax(dim=1).item()
            next_state, reward, done= env.step(action)
            ep_reward+= reward

            with torch.no_grad():
                s_next= torch.FloatTensor(next_state).unsqueeze(0)
                next_q_values= policy_net(s_next)
                target_value= reward + (0 if done else gamma* next_q_values.max(dim=1)[0].item())

            s= torch.FloatTensor(state).unsqueeze(0)
            q_values= policy_net(s)
            q_current= q_values[0, action]
            loss= F.mse_loss(q_current, torch.FloatTensor([target_value]))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            state= next_state
        epsilon= max(epsilon_min, epsilon*epsilon_decay)
        rewards_per_episode.append({"Episode": ep+1, "Total Reward": ep_reward})

    return pd.DataFrame(rewards_per_episode), policy_net

def simulate_dqn_trades(policy_net, df):
    env= TradingEnv(df)
    state= env.reset()
    all_actions=[]
    day_idx=0

    done=False
    while not done:
        with torch.no_grad():
            s= torch.FloatTensor(state).unsqueeze(0)
            q_values= policy_net(s)
            action= q_values.argmax(dim=1).item()

        if action==1:
            action_name= "Buy"
        elif action==2:
            action_name= "Sell"
        else:
            action_name= "Hold"

        if day_idx< len(df):
            date_here= df.index[day_idx]
            price_here= df.loc[date_here,'close']
            all_actions.append((date_here, action_name, price_here))

        next_state, reward, done= env.step(action)
        state= next_state
        day_idx+=1
    return all_actions

def plot_trades_chart(df, all_actions):
    fig, ax= plt.subplots(figsize=(12,6))
    ax.plot(df.index, df['close'], label="Close Price")
    used_labels= set()

    for (date, action_name, price) in all_actions:
        if action_name=="Buy":
            marker,color= "^","green"
        elif action_name=="Sell":
            marker,color= "v","red"
        else:
            marker,color= ".","blue"

        lbl= action_name if action_name not in used_labels else None
        used_labels.add(action_name)
        ax.scatter(date, price, marker=marker, color=color, s=100, label=lbl)

    ax.legend()
    ax.set_title("DQN Trades on Price Chart (All Actions Plotted)")
    buf= io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight")
    buf.seek(0)
    plt.close(fig)
    return Image.open(buf)

def run_dqn_csv(csv_data_path="full_data.csv", episodes=5):
    try:
        df= pd.read_csv(csv_data_path, parse_dates=True, index_col=0)
        if 'close' not in df.columns:
            raise gr.Error("No 'close' column in CSV. Please ensure CSV has the necessary columns.")
        rewards_df, policy_net= train_dqn_from_df(df, episodes=episodes)
        all_actions= simulate_dqn_trades(policy_net, df)
        print("=== DQN Actions Day by Day ===")
        for item in all_actions:
            print(item)
        print("================================")
        trade_plot= plot_trades_chart(df, all_actions)
        return rewards_df, trade_plot
    except Exception as e:
        raise gr.Error(f"An error occurred in the DQN module: {e}")

In [56]:
# Gradio UI
default_end_date= datetime.now().strftime("%Y-%m-%d")
default_start_date= (datetime.now()-timedelta(days=365)).strftime("%Y-%m-%d")

chart_interface= gr.Interface(
    fn= generate_plot,
    inputs=[
        gr.Textbox(label="Ticker", value="SPY"),
        gr.Textbox(label="Start Date (YYYY-MM-DD)", value=default_start_date),
        gr.Textbox(label="End Date (YYYY-MM-DD)",   value=default_end_date),

        # Weighted scoreboard
        gr.Slider(minimum=0, maximum=2, step=0.1, value=1.0, label="ZLMA Weight"),
        gr.Slider(minimum=0, maximum=2, step=0.1, value=1.0, label="RSI Weight"),
        gr.Slider(minimum=0, maximum=2, step=0.1, value=1.0, label="MCDX Weight"),
        gr.Slider(minimum=0, maximum=2, step=0.1, value=1.0, label="DSS Weight"),
        gr.Slider(minimum=0, maximum=2, step=0.1, value=1.0, label="Z-Score Weight"),
        gr.Slider(minimum=0, maximum=2, step=0.1, value=0.0, label="Momentum Grade Weight"),
        gr.Slider(minimum=0, maximum=2, step=0.1, value=1.0, label="Momentum Direction Weight"),
        gr.Slider(minimum=0, maximum=2, step=0.1, value=0.0, label="Momentum State Weight"),

        # Two MACD weights
        gr.Slider(minimum=0, maximum=2, step=0.1, value=1.0, label="ZeroLag MACD Weight"),
        gr.Slider(minimum=0, maximum=2, step=0.1, value=1.0, label="Basic MACD Weight"),
    ],
    outputs=[
        gr.Image(type="pil", label="Chart"),
        gr.Dataframe(label="Current Indicator Status"),
        gr.Dataframe(label="Historical Signals"),
        gr.Dataframe(label="Trade Log")
    ],
    title="Oberon Trading Bot (ZeroLag + Basic MACD, Weighted Scoreboard + Historical Trade Table)",
    description=(
        "Displays a multi-panel chart including candlesticks, EMAs, ZLMA, RSI, Bressert, MCDX, ZeroLag MACD, Basic MACD, and momentum.\n"
        "You can adjust the weighting for each indicator in the scoreboard. The final row of the Current Indicator Status shows the ScoreSum and OverallTrade.\n\n"
        "Outputs:\n"
        "1) Chart\n2) Current Indicator Status (weighted scoreboard)\n3) Historical Signals\n"
        "4) Trade Log (calculated using user-specified weights, flipping from Buy to Sell day by day)."
    )
)

with gr.Blocks() as demo:
    with gr.Tab("Chart + Trade Log"):
        chart_interface.render()

    with gr.Tab("Save Data"):
        gr.Markdown("### Save Historical Data & Signals to CSV")
        with gr.Row():
            sd_ticker= gr.Textbox(label="Ticker", value="SPY")
            sd_start=  gr.Textbox(label="Start Date", value=default_start_date)
            sd_end=    gr.Textbox(label="End Date",   value=default_end_date)
        data_csv_path=    gr.Textbox(label="Data CSV Filename",    value="full_data.csv")
        signals_csv_path= gr.Textbox(label="Signals CSV Filename", value="signals_data.csv")
        btn_save= gr.Button("Save to CSV")
        save_output_msg= gr.Markdown()

        def save_data_wrapper(ticker, start, end, datafile, sigfile):
            return save_historical_data(ticker, start, end, datafile, sigfile)

        btn_save.click(
            fn= save_data_wrapper,
            inputs=[sd_ticker, sd_start, sd_end, data_csv_path, signals_csv_path],
            outputs= save_output_msg
        )

    with gr.Tab("DQN Training"):
        gr.Markdown("### Train DQN from Saved CSV & Show Trades")
        csv_input= gr.Textbox(label="CSV file path", value="full_data.csv", info="Path to CSV created above.")
        episodes_input= gr.Slider(label="Episodes", value=5, minimum=1, maximum=200, step=1)
        run_dqn_btn= gr.Button("Train DQN & Show Trades")
        dqn_rewards_out= gr.Dataframe(label="Episode Rewards")
        dqn_tradeplot_out= gr.Image(type="pil", label="DQN Trades Chart")

        run_dqn_btn.click(
            fn= run_dqn_csv,
            inputs= [csv_input, episodes_input],
            outputs=[dqn_rewards_out, dqn_tradeplot_out]
        )

if __name__=="__main__":
    demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cd5600dd1314acc187.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Changelog Version: 1.2.2
- Fixed trade table
- Minor formatting

### 1.2.1
- Added trade table
- Added user adjustable weights
- Added basic MACD
- Added trade signals